## For Landsat Atmospheric Correction

Reference: [Landsat Level-1 Data Product](https://www.usgs.gov/landsat-missions/using-usgs-landsat-level-1-data-product)

In [3]:
# !pip install rasterio

In [6]:
import rasterio
import numpy as np
import os
import re
import glob


In [10]:

def parse_mtl(mtl_file):
    """
    Parse the MTL file to extract relevant metadata for correction.
    Parameters:
        mtl_file (str): Path to the MTL file.
    Returns:
        dict: Parsed metadata.
    """
    metadata = {}
    with open(mtl_file, 'r') as f:
        for line in f:
            line = line.strip()
            match = re.match(r'(\w+)\s=\s(.*)', line)
            if match:
                key, value = match.groups()
                value = value.replace('"', '')
                metadata[key] = float(value) if re.match(r'^-?\d+(\.\d+)?$', value) else value
    return metadata

def toa_radiance(band, radiance_mult, radiance_add):
    """
    Convert DN to TOA Radiance.
    Parameters:
        band (numpy.ndarray): Raster data for a single band.
        radiance_mult (float): Radiance multiplicative scaling factor.
        radiance_add (float): Radiance additive scaling factor.
    Returns:
        numpy.ndarray: TOA Radiance.
    """

    band = band.astype(np.float32)
    return band * radiance_mult + radiance_add

def toa_reflectance(band, reflectance_mult, reflectance_add, sun_elevation):
    """
    Convert DN to TOA Reflectance.
    Parameters:
        band (numpy.ndarray): Raster data for a single band.
        reflectance_mult (float): Reflectance multiplicative scaling factor.
        reflectance_add (float): Reflectance additive scaling factor.
        sun_elevation (float): Sun elevation angle in degrees.
    Returns:
        numpy.ndarray: TOA Reflectance.
    """
    band = band.astype(np.float32) # here as well - as the raster has float32
    reflectance = band * reflectance_mult + reflectance_add
    return reflectance / np.sin(np.deg2rad(sun_elevation))


In [11]:
input_dir = r"C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2"
output_dir = r"C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\output"
os.makedirs(output_dir, exist_ok=True)

mtl_file = next((os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith("MTL.txt")), None)
if not mtl_file:
    raise FileNotFoundError("No MTL file found in the input directory!")

metadata = parse_mtl(mtl_file)

sun_elevation = metadata["SUN_ELEVATION"]
band_numbers = [1, 2, 3, 4, 5, 6, 7]  # You can write depending ion the bands you need to use - the #s

In [13]:

for band_number in band_numbers:
    # Search for files that end exactly with "T2_B{band_number}.TIF"
    search_pattern = os.path.join(input_dir, f"*T2_B{band_number}.TIF")
    matching_files = glob.glob(search_pattern)
    
    if not matching_files:
        print(f"No file found for Band {band_number} using pattern {search_pattern}, skipping.")
        continue

# Use the first matching file (should be unique for each band)
    band_file = matching_files[0]
    print(f"Processing file: {band_file}")

    with rasterio.open(band_file) as src:
        band = src.read(1)
        profile = src.profile

        radiance_mult = float(metadata.get(f"RADIANCE_MULT_BAND_{band_number}", 0)) # float to avoid the datatype error
        radiance_add = float(metadata.get(f"RADIANCE_ADD_BAND_{band_number}", 0))
        reflectance_mult = float(metadata.get(f"REFLECTANCE_MULT_BAND_{band_number}", 0))
        reflectance_add = float(metadata.get(f"REFLECTANCE_ADD_BAND_{band_number}", 0))

        if radiance_mult and radiance_add:
            toa_rad = toa_radiance(band, radiance_mult, radiance_add)
            profile.update(dtype='float32')
            output_file = os.path.join(output_dir, f"toa_radiance_band_{band_number}.tif")
            with rasterio.open(output_file, 'w', **profile) as dst:
                dst.write(toa_rad.astype('float32'), 1)
            print(f"TOA Radiance for band {band_number} saved to: {output_file}")

        if reflectance_mult and reflectance_add:
            toa_refl = toa_reflectance(band, reflectance_mult, reflectance_add, sun_elevation)
            profile.update(dtype='float32')
            output_file = os.path.join(output_dir, f"toa_reflectance_band_{band_number}.tif")
            with rasterio.open(output_file, 'w', **profile) as dst:
                dst.write(toa_refl.astype('float32'), 1)
            print(f"TOA Reflectance for band {band_number} saved to: {output_file}")

print("Radiometric correction completed!")


Processing file: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\LE07_L1GT_029030_20240114_20240211_02_T2_B1.TIF
TOA Radiance for band 1 saved to: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\output\toa_radiance_band_1.tif
TOA Reflectance for band 1 saved to: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\output\toa_reflectance_band_1.tif
Processing file: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\LE07_L1GT_029030_20240114_20240211_02_T2_B2.TIF
TOA Radiance for band 2 saved to: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\output\toa_radiance_band_2.tif
TOA Reflectance for band 2 saved to: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\output\toa_reflectance_band_2.tif
Processing file: C:\Users\Khizer Zakir\Downloads\LE07_L1GT_029030_20240114_20240211_02_T2\LE07_L1GT_029030_20240114_20240211_02_T2_B3.TIF
TOA Radiance for band 